In [1]:

import numpy as np
import random
import os
import pickle
import yaml
from easydict import EasyDict

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch_geometric.data import Data, Dataset
from torch_geometric.transforms import Compose
from rdkit import Chem
from confgf import models,dataset, runner, utils

In [9]:

# your account is:
# user: yitaocai
# pwd: UJCQqv77
# ssh -p 5792 yitaocai@student07.ai-lab.jku.at

In [10]:
config_path = "/system/user/yitaocai/PracticalAI/config/qm9_default.yml"
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)
config = EasyDict(config)

NameError: name 'EasyDict' is not defined

In [3]:

import torch
print(torch.cuda.is_available()) 


True


In [8]:
if config.test.output_path is not None:
    config.test.output_path = os.path.join(config.test.output_path, config.model.name)
    if not os.path.exists(config.test.output_path):
        os.makedirs(config.test.output_path)

In [1]:
!export CUDA_VISIBLE_DEVICES=0

In [11]:

# export CUDA_VISIBLE_DEVICES=1 before work
# gpus = list(filter(lambda x: x is not None, config.train.gpus))
# assert torch.cuda.device_count() >= len(gpus), 'do you set the gpus in config correctly?'


# print("Let's use", len(gpus), "GPUs!")
  
# config.train.device = device
# config.train.gpus = gpus

Using device cpu as main device


In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device %s as main device" % device)  
np.random.seed(config.train.seed)
random.seed(config.train.seed)
torch.manual_seed(config.train.seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(config.train.seed)        
    torch.cuda.manual_seed_all(config.train.seed)
torch.backends.cudnn.benchmark = True
print('set seed for random, numpy and torch')

set seed for random, numpy and torch


In [8]:
load_path = os.path.join(config.data.base_path, '%s_processed' % config.data.dataset)
print('loading data from %s' % load_path)

train_data = []
val_data = []
test_data = []

if config.data.test_set is not None:
    with open(os.path.join(load_path, config.data.test_set), "rb") as fin:
        test_data = pickle.load(fin)             
else:
    raise ValueError("do you set the test data ?")              

loading data from /Users/mindy/百度云同步盘/StudyResearch/SeminarPraticalAI/qm9_processed


In [10]:
print('train size : %d  ||  val size: %d  ||  test size: %d ' % (len(train_data), len(val_data), len(test_data)))
print('loading data done!')

transform = Compose([
    utils.AddHigherOrderEdges(order=config.model.order),
    utils.AddEdgeLength(),
    utils.AddPlaceHolder(),
    utils.AddEdgeName()
])
train_data = dataset.GEOMDataset(data=train_data, transform=transform)
val_data = dataset.GEOMDataset(data=val_data, transform=transform)
test_data = dataset.GEOMDataset_PackedConf(data=test_data, transform=transform)
print('len of test data: %d' % len(test_data))

model = models.DistanceScoreMatch(config)

#optimizer = utils.get_optimizer(config.train.optimizer, model)    
optimizer = None
#scheduler = utils.get_scheduler(config.train.scheduler, optimizer)
scheduler = None

train size : 0  ||  val size: 0  ||  test size: 24068 
loading data done!
got 200 molecules with 24068 confs
len of test data: 200


In [11]:
solver = runner.DefaultRunner(train_data, val_data, test_data, model, optimizer, scheduler, config)

assert config.test.init_checkpoint is not None
solver.load(config.test.init_checkpoint, epoch=config.test.epoch)





Load checkpoint from /Users/mindy/百度云同步盘/StudyResearch/SeminarPraticalAI/ConfGF-main/config/confgf_checkpoint/checkpoint284


In [16]:
smilesfile ="/system/user/yitaocai/PracticalAI/tox21_random.txt"
generator = "ConfGF"
if smilesfile is not None:
    with open(smilesfile, "r") as f:
        lines = f.readlines()
    smileslist = [line.strip().split("\t") for line in lines]
    confdata = {}
    for smiles,key in smileslist[0:1000]:

        threedconf =solver.generate_samples_from_smiles(smiles, generator, \
                                            num_repeat=1, keep_traj=True,
                                            out_path=None)
        confdata[key] = threedconf
    out_path = config.test.output_path
    if out_path is not None:
        with open(os.path.join(out_path, 'CONFDATA1.pkl'), "wb") as fout:
            pickle.dump(confdata, fout)
            print('save generated %s samples to %s done!' % (generator, out_path))

    # print('pos generation of %s done' % return_data.smiles) 

Sampling positions: 100%|██████████| 50/50 [01:20<00:00,  1.61s/it]


pos generation of Nc1cc2nc3cc(N)ccc3cc2cc1.[Cl-].c12c([n+](C)c3c(c1)ccc(N)c3)cc(N)cc2 done


Sampling positions: 100%|██████████| 50/50 [00:48<00:00,  1.03it/s]


pos generation of c12cc(Br)c([O-])c(Br)c1oc1c(Br)c(=O)c(Br)cc-1c2-c1ccccc1C([O-])=O.[Na+].[Na+] done


Sampling positions:   6%|▌         | 3/50 [00:12<03:21,  4.29s/it]


KeyboardInterrupt: 

In [1]:
len(confdata)

NameError: name 'confdata' is not defined